### 1. Initial Setup and Data Loading

Start by importing necessary libraries and loading the raw data collected from the API into raw_df.
Error handling ensures catching missing files.

In [357]:
# Import standard libraries
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time
import warnings
warnings.filterwarnings('ignore')

# Load the data
try:
    raw_df = pd.read_csv('ev_charging_data.csv')
except FileNotFoundError:
    print("CSV file not found.")
    exit()

### 2. Initial Data Assessement

Basic exploration to understand the dataset's structure, size and content (from 01_data_aquistion) before any transformation.

In [358]:
print(f"Dataset shape: {raw_df.shape}")
print(f"Total number of columns: {len(raw_df.columns)}")

Dataset shape: (863, 83)
Total number of columns: 83


In [359]:
# Check column names to consider what to keep
print(f"Original colum names: {raw_df.columns.tolist()}")

Original colum names: ['UserComments', 'PercentageSimilarity', 'MediaItems', 'IsRecentlyVerified', 'DateLastVerified', 'ID', 'UUID', 'ParentChargePointID', 'DataProviderID', 'DataProvidersReference', 'OperatorID', 'OperatorsReference', 'UsageTypeID', 'UsageCost', 'Connections', 'NumberOfPoints', 'GeneralComments', 'DatePlanned', 'DateLastConfirmed', 'StatusTypeID', 'DateLastStatusUpdate', 'MetadataValues', 'DataQualityLevel', 'DateCreated', 'SubmissionStatusTypeID', 'DataProvider.WebsiteURL', 'DataProvider.Comments', 'DataProvider.DataProviderStatusType.IsProviderEnabled', 'DataProvider.DataProviderStatusType.ID', 'DataProvider.DataProviderStatusType.Title', 'DataProvider.IsRestrictedEdit', 'DataProvider.IsOpenDataLicensed', 'DataProvider.IsApprovedImport', 'DataProvider.License', 'DataProvider.DateLastImported', 'DataProvider.ID', 'DataProvider.Title', 'OperatorInfo.WebsiteURL', 'OperatorInfo.Comments', 'OperatorInfo.PhonePrimaryContact', 'OperatorInfo.PhoneSecondaryContact', 'Operato

### 3. Core Column Section and Renaming

We select only relevant columns and rename them to for clearer understanding, creating core_df wich will be the main working dataframe.

In [360]:
# Define the core columns we need for analysis
core_columns = [
    # Station ID
    'ID',
    
    # Location & Geographic Data
    'AddressInfo.Latitude',
    'AddressInfo.Longitude',
    'AddressInfo.Town',
    'AddressInfo.StateOrProvince',
    'AddressInfo.Postcode',
    'AddressInfo.Country.Title',
    'AddressInfo.AccessComments',
    
    # Charger Specifications
    'Connections',               # Charger types and power levels
    'NumberOfPoints',            # How many charging points available
    
    # Operational Status
    'StatusType.IsOperational',
    'DateLastVerified',
    'DateCreated',
    
    # Pricing & Access
    'UsageCost',
    'UsageType.IsPayAtLocation',
    'UsageType.IsMembershipRequired',
    
    # Operator Information
    'OperatorInfo.Title',
]

# Create a mapping dictionary for cleaner column names
column_rename_map = {
    # Location & Geographic Data
    'AddressInfo.Latitude': 'latitude',
    'AddressInfo.Longitude': 'longitude',
    'AddressInfo.Town': 'city',
    'AddressInfo.StateOrProvince': 'state',
    'AddressInfo.Postcode': 'postal_code',
    'AddressInfo.Country.Title': 'country',
    'AddressInfo.AccessComments': 'access_comments',
    
    # Charger Specifications
    'Connections': 'connections',
    'NumberOfPoints': 'num_charging_points',
    
    # Operational Status
    'StatusType.IsOperational': 'is_operational',
    'DateLastVerified': 'last_verified_date',
    'DateCreated': 'creation_date',
    
    # Pricing & Access
    'UsageCost': 'usage_cost',
    'UsageType.IsPayAtLocation': 'is_pay_at_location',
    'UsageType.IsMembershipRequired': 'is_membership_required',
    
    # Operator Information
    'OperatorInfo.Title': 'operator',
}

In [361]:
# Check which core columns exist in our dataset
available_columns = [col for col in core_columns if col in df.columns]
missing_columns = [col for col in core_columns if col not in df.columns]
if missing_columns:
    print("Warning: Missing columns:", missing_columns)
df_core = raw_df[available_columns].copy().rename(columns=column_rename_map)

print(f"Available core columns: {len(available_columns)}")
print(f"Columns after rename: {df_core.columns.tolist()}")
print(f"Missing columns: {missing_columns}")

Available core columns: 17
Columns after rename: ['ID', 'latitude', 'longitude', 'city', 'state', 'postal_code', 'country', 'access_comments', 'connections', 'num_charging_points', 'is_operational', 'last_verified_date', 'creation_date', 'usage_cost', 'is_pay_at_location', 'is_membership_required', 'operator']
Missing columns: []


In [362]:
df_core.head()

,ID,latitude,longitude,city,state,postal_code,country,access_comments,connections,num_charging_points,is_operational,last_verified_date,creation_date,usage_cost,is_pay_at_location,is_membership_required,operator
0,313310,47.677106,19.667077,Hatvan,Heves,3000.0,Hungary,NaN,"[{'ID': 603425, 'ConnectionTypeID': 33, 'Conne...",2.0,True,2025-06-16T19:34:00Z,2025-06-04T15:18:00Z,NaN,False,True,Mobiliti.hu
1,313307,47.561080,19.049120,NaN,NaN,NaN,Hungary,NaN,"[{'ID': 603422, 'ConnectionTypeID': 33, 'Conne...",12.0,True,2025-06-07T13:14:00Z,2025-06-04T13:10:00Z,202 HUF/kWh,False,True,Tesla (including non-tesla)
2,313306,47.383440,19.026830,NaN,NaN,NaN,Hungary,NaN,"[{'ID': 603421, 'ConnectionTypeID': 33, 'Conne...",20.0,True,2025-06-07T13:14:00Z,2025-06-04T13:09:00Z,202 HUF/kWh,False,True,Tesla (including non-tesla)
3,313305,47.149650,18.431330,NaN,NaN,NaN,Hungary,NaN,"[{'ID': 603420, 'ConnectionTypeID': 33, 'Conne...",12.0,True,2025-06-07T13:09:00Z,2025-06-04T13:07:00Z,202 HUF/kWh,False,True,Tesla (including non-tesla)
4,312979,46.926144,17.837953,Aszófő,NaN,8241.0,Hungary,NaN,"[{'ID': 603068, 'ConnectionTypeID': 33, 'Conne...",2.0,True,2025-06-11T14:02:00Z,2025-06-01T16:50:00Z,Day: 129 Ft/kWh | Night: 249 Ft/kWh,False,True,EV Direct


### 4. Geographic Data Celaning


This section standardizes Hungarian county names and handles missing geographic data through reverse geocoding when coordinates are available

In [363]:
# Geographic data information
print("Geographic Data Info:")
print("Latitude/Longitude missing values:")
print(f"Latitude missing: {df_core['latitude'].isna().sum()} ({df_core['latitude'].isna().mean()*100:.1f}%)")
print(f"Longitude missing: {df_core['longitude'].isna().sum()} ({df_core['longitude'].isna().mean()*100:.1f}%)")

Geographic Data Info:
Latitude/Longitude missing values:
Latitude missing: 0 (0.0%)
Longitude missing: 0 (0.0%)


In [364]:
# Location data information
print("Location Data Cleaning")
print("Location missing values:")
print(f"State/Province missing: {df_core['state'].isna().sum()} ({df_core['state'].isna().mean()*100:.1f}%)")
print(f"Country missing: {df_core['country'].isna().sum()} ({df_core['country'].isna().mean()*100:.1f}%)")
print(f"City missing: {df_core['city'].isna().sum()} ({df_core['city'].isna().mean()*100:.1f}%)")
print(f"Postal Code missing: {df_core['postal_code'].isna().sum()} ({df_core['postal_code'].isna().mean()*100:.1f}%)")

Location Data Cleaning
Location missing values:
State/Province missing: 281 (32.6%)
Country missing: 0 (0.0%)
City missing: 8 (0.9%)
Postal Code missing: 104 (12.1%)


In [365]:
# Check State/province data
df_core['state'].value_counts()


state
Budapest                        114
Pest megye                       62
Fejér megye                      41
Veszprém megye                   36
Bács-Kiskun megye                24
Győr-Moson-Sopron megye          24
Zala megye                       20
Borsod-Abaúj-Zemplén megye       18
Hajdú-Bihar megye                14
Baranya megye                    14
Somogy megye                     14
Heves megye                      13
Csongrád megye                   13
Pest                             13
Vas megye                        11
Komárom-Esztergom megye          11
Békés megye                      10
Tolna megye                       9
Jász-Nagykun-Szolnok megye        9
Közép-Magyarország                8
Győr-Moson-Sopron                 8
Szabolcs-Szatmár-Bereg megye      8
Great Plain and North             7
Vas                               6
Baranya                           6
Magyarország                      6
Tolna                             5
Zala                  

In [366]:
# Replace missing values in State/Province with 'Unknown'
df_core['state'] = df_core['state'].fillna('Unknown')

In [367]:
# Check if there are any missing values in the State/Province column
df_core['state'].isna().any()

np.False_

In [368]:
# Check unique values of state/province
df_core['state'].unique()

array(['Heves ', 'Unknown', 'Fejér', 'Pest megye', 'Borsod-Abaúj-Zemplén',
       'Heves vármegye', 'Magyarország', 'Pest megye ',
       'Győr-Moson-Sopron', 'Veszprém megye', 'Fejér megye', 'Budapest',
       'Komárom-Esztergom megye', 'Zala megye', 'Bács-Kiskun megye',
       'Vas megye', 'Borsod-Abaúj-Zemplén megye', 'Közép-Magyarország',
       'Somogy megye', 'Heves megye', 'Szabolcs-Szatmár-Bereg megye',
       'Békés megye', 'Jász-Nagykun Szolnok megye', 'Transdanubia',
       'Baranya', ' Szabolcs-Szatmár-Bereg', 'Tolna megye',
       'Great Plain and North', 'Bács-Kiskun', 'Vas', 'Baranya megye',
       'Jász-Nagykun-Szolnok', 'Jász-Nagykun-Szolnok megye',
       'Csongrád-Csanád megye', 'Győr-Moson-Sopron megye', 'Pest', 'Zala',
       'Tolna', 'Borsod', 'Heves', 'Komárom-Esztergom', 'Hajdú-Bihar',
       'Szabolcs-Szatmár-Bereg', 'Somogy', 'Nógrád', 'Hajdú-Bihar megye',
       'Fejér Megye', 'Csongrád megye', 'Heves-megye', 'Veszprém',
       'Sormas', 'Zala-megye', 'Komita

In [369]:
def standardize_counties():
    """
    Create a mapping of Hungarian counties to their standard names
    Based on official Hungarian administrative divisions (19 counties + Budapest)
    """
    county_mapping = {
        # Standard county names
        'Budapest': 'Budapest',
        'Pest': 'Pest',
        'Fejér': 'Fejér',
        'Komárom-Esztergom': 'Komárom-Esztergom',
        'Veszprém': 'Veszprém',
        'Győr-Moson-Sopron': 'Győr-Moson-Sopron',
        'Vas': 'Vas',
        'Zala': 'Zala',
        'Baranya': 'Baranya',
        'Somogy': 'Somogy',
        'Tolna': 'Tolna',
        'Bács-Kiskun': 'Bács-Kiskun',
        'Csongrád-Csanád': 'Csongrád-Csanád',
        'Jász-Nagykun-Szolnok': 'Jász-Nagykun-Szolnok',
        'Hajdú-Bihar': 'Hajdú-Bihar',
        'Szabolcs-Szatmár-Bereg': 'Szabolcs-Szatmár-Bereg',
        'Borsod-Abaúj-Zemplén': 'Borsod-Abaúj-Zemplén',
        'Heves': 'Heves',
        'Nógrád': 'Nógrád',
        'Békés': 'Békés',
        
        # Variations that need to be mapped
        'Pest megye': 'Pest',
        'Pest megye ': 'Pest',
        'Fejér megye': 'Fejér',
        'Fejér Megye': 'Fejér',
        'Komárom-Esztergom megye': 'Komárom-Esztergom',
        'Veszprém megye': 'Veszprém',
        'Győr-Moson-Sopron megye': 'Győr-Moson-Sopron',
        'Vas megye': 'Vas',
        'Zala megye': 'Zala',
        'Zala-megye': 'Zala',
        'Komitat Zala': 'Zala',
        'Baranya megye': 'Baranya',
        'Somogy megye': 'Somogy',
        'Tolna megye': 'Tolna',
        'Bács-Kiskun megye': 'Bács-Kiskun',
        'Csongrád-Csanád megye': 'Csongrád-Csanád',
        'Csongrád megye': 'Csongrád-Csanád',  # Old name, merged in 2020
        'Jász-Nagykun-Szolnok megye': 'Jász-Nagykun-Szolnok',
        'Jász-Nagykun Szolnok megye': 'Jász-Nagykun-Szolnok',
        'Hajdú-Bihar megye': 'Hajdú-Bihar',
        'Szabolcs-Szatmár-Bereg megye': 'Szabolcs-Szatmár-Bereg',
        ' Szabolcs-Szatmár-Bereg': 'Szabolcs-Szatmár-Bereg',
        'Borsod-Abaúj-Zemplén megye': 'Borsod-Abaúj-Zemplén',
        'Borsod': 'Borsod-Abaúj-Zemplén',
        'Heves megye': 'Heves',
        'Heves ': 'Heves',
        'Heves vármegye': 'Heves',
        'Heves-megye': 'Heves',
        'Nógrád megye': 'Nógrád',
        'Békés megye': 'Békés',

         # ISO codes and abbreviations
        'HU-BU': 'Budapest',  # ISO-style code for Budapest
        'HU-PE': 'Pest',      # ISO-style code for Pest
        'HU-FE': 'Fejér',     # ISO-style code for Fejér
        'HU-KE': 'Komárom-Esztergom',
        'HU-VE': 'Veszprém',
        'HU-GS': 'Győr-Moson-Sopron',
        'HU-VA': 'Vas',
        'HU-ZA': 'Zala',
        'HU-BA': 'Baranya',
        'HU-SO': 'Somogy',
        'HU-TO': 'Tolna',
        'HU-BK': 'Bács-Kiskun',
        'HU-CS': 'Csongrád-Csanád',
        'HU-JN': 'Jász-Nagykun-Szolnok',
        'HU-HB': 'Hajdú-Bihar',
        'HU-SZ': 'Szabolcs-Szatmár-Bereg',
        'HU-BZ': 'Borsod-Abaúj-Zemplén',
        'HU-HE': 'Heves',
        'HU-NO': 'Nógrád',
        'HU-BE': 'Békés',
        
        # Invalid/regional names that need geocoding
        'Magyarország': None,  # "Hungary" - too broad
        'Hungary': None,
        'Közép-Magyarország': None,  # "Central Hungary" - regional grouping
        'Transdanubia': None,  # Regional grouping
        'Great Plain and North': None,  # Regional grouping
        'Unknown': None,
        'Sormas': None,  # Likely a data error
    }
    
    return county_mapping
        

In [370]:
def reverse_geocode_location(lat, lon, max_retries=3):
    """
    Given latitude and longitude, attempts up to `max_retries` to reverse-geocode
    and return a dict with keys:
       - 'county': county/state or province name (cleaned of suffixes)
       - 'city': city/town/village name
       - 'postal_code': postal code (if present)
    If any field can’t be found, its value will be None.
    """

    # 1) skip NaN values
    if pd.isna(lat) or pd.isna(lon):
        return {'county': None, 'city': None, 'postal_code': None}

    # 2) Try reverse geocoding
    try:
        lat, lon = float(lat), float(lon)
    except Exception:
        return {'county': None, 'city': None, 'postal_code': None}
    
    geolocator = Nominatim(user_agent = "ev_charging_analysis", timeout=10)
    reverse_geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1, max_retries=5, error_wait_seconds=2.0)

    for attempt in range(1, max_retries + 1):
        try:
            location = reverse_geocode((lat, lon), exactly_one=True, addressdetails=True)
            if not location or 'address' not in location.raw:
                return {'county': None, 'city': None, 'postal_code': None}

            addr = location.raw['address']

            # Extract county/state/province
            county = None
            for key in ('county', 'state', 'state_district', 'ISO3166-2-lvl6'):
                if addr.get(key):
                    county = addr[key].strip()
                    # strip common suffixes
                    for suf in [' megye', ' vármegye', ' County', ' Province']:
                        if county.endswith(suf):
                            county = county[:-len(suf)].strip()
                    break

            # Extract city-like field
            city = None
            for key in ('city', 'town', 'village', 'municipality', 'hamlet'):
                if addr.get(key):
                    city = addr[key].strip()
                    break

            # Extract postal code
            postal_code = addr.get('postcode', None)
            if postal_code:
                postal_code = postal_code.strip()

            return {
                'county': county,
                'city': city,
                'postal_code': postal_code
            }

        except Exception as e:
            # On failure, retry after exponential backoff up to max_retries
            if attempt < max_retries:
                wait = 2 ** (attempt - 1)
                time.sleep(wait)
                continue
            else:
                # Return None after last retry
                return {'county': None, 'city': None, 'postal_code': None}

In [371]:
def clean_geographic_data(df_core):
    """
    1) Standardize any state/county strings via a hard-coded mapping.
    2) Identify rows still missing county, city or postal_code.
    3) For those rows, call reverse_geocode_location to fill in all three.
    4) Drop any rows that remain unresolved for county.
    5) Rename/move columns so that the result has:
         - 'county', 'city', 'postal_code' cleaned
         - original latitude/longitude preserved
    """
    print("Starting geographic data cleaning...")
    df_core = df_core.copy()

    # 1) County mapping
    county_map = standardize_counties()
    df_core['county'] = df_core['state'].map(county_map)

    # 2) Use existing city/postal_code where present
    df_core['city'] = df_core['city'].astype(object)
    df_core['postal_code'] = df_core['postal_code'].astype(object)

    # 3) Find rows needing any reverse-geocode fill
    needs_geo_county = df_core[['county']].isna().any(axis=1)
    needs_geo_city = df_core[['city']].isna().any(axis=1)
    needs_geo_postalcode = df_core[['postal_code']].isna().any(axis=1)
    needs_geo = df_core[['county','city','postal_code']].isna().any(axis=1)
    print(f"Counties needing reverse-geocoding: {needs_geo_county.sum()}")
    print(f"Cities needing reverse-geocoding: {needs_geo_city.sum()}")
    print(f"Postal codes needing reverse-geocoding: {needs_geo_postalcode.sum()}")
    print(f"Rows needing reverse-geocoding: {needs_geo.sum()}")

    # 4) Loop over those and fill
    for idx in df_core[needs_geo].index:
        lat = df_core.at[idx, 'latitude']
        lon = df_core.at[idx, 'longitude']
        result = reverse_geocode_location(lat, lon)

        # Fill missing only
        if pd.isna(df_core.at[idx, 'county']) and result['county']:
            # After mapping through standard_map if possible
            std = county_map.get(result['county'], result['county'])
            df_core.at[idx, 'county'] = std

        if pd.isna(df_core.at[idx, 'city']) and result['city']:
            df_core.at[idx, 'city'] = result['city']

        if pd.isna(df_core.at[idx, 'postal_code']) and result['postal_code']:
            df_core.at[idx, 'postal_code'] = result['postal_code']

        # small pause
        time.sleep(0.1)

    # 5) Drop rows where county still missing
    unresolved = df_core['county'].isna()
    if unresolved.sum() > 0:
        print(f"Dropping {unresolved.sum()} rows with unresolved county")
        df_core = df_core[~unresolved]

    # 6) Clean up: drop original 'state' column
    df_core = df_core.drop(columns=['state'])
    print("Geographic cleaning complete.")
    return df_core

In [372]:
geo_clean_df = clean_geographic_data(df_core)

Starting geographic data cleaning...
Counties needing reverse-geocoding: 308
Cities needing reverse-geocoding: 8
Postal codes needing reverse-geocoding: 104
Rows needing reverse-geocoding: 320
Geographic cleaning complete.


In [373]:
geo_clean_df.head()

,ID,latitude,longitude,city,postal_code,country,access_comments,connections,num_charging_points,is_operational,last_verified_date,creation_date,usage_cost,is_pay_at_location,is_membership_required,operator,county
0,313310,47.677106,19.667077,Hatvan,3000.0,Hungary,NaN,"[{'ID': 603425, 'ConnectionTypeID': 33, 'Conne...",2.0,True,2025-06-16T19:34:00Z,2025-06-04T15:18:00Z,NaN,False,True,Mobiliti.hu,Heves
1,313307,47.561080,19.049120,Budapest,1033,Hungary,NaN,"[{'ID': 603422, 'ConnectionTypeID': 33, 'Conne...",12.0,True,2025-06-07T13:14:00Z,2025-06-04T13:10:00Z,202 HUF/kWh,False,True,Tesla (including non-tesla),Budapest
2,313306,47.383440,19.026830,Szigetszentmiklós,2310,Hungary,NaN,"[{'ID': 603421, 'ConnectionTypeID': 33, 'Conne...",20.0,True,2025-06-07T13:14:00Z,2025-06-04T13:09:00Z,202 HUF/kWh,False,True,Tesla (including non-tesla),Pest
3,313305,47.149650,18.431330,Székesfehérvár,8000,Hungary,NaN,"[{'ID': 603420, 'ConnectionTypeID': 33, 'Conne...",12.0,True,2025-06-07T13:09:00Z,2025-06-04T13:07:00Z,202 HUF/kWh,False,True,Tesla (including non-tesla),Fejér
4,312979,46.926144,17.837953,Aszófő,8241.0,Hungary,NaN,"[{'ID': 603068, 'ConnectionTypeID': 33, 'Conne...",2.0,True,2025-06-11T14:02:00Z,2025-06-01T16:50:00Z,Day: 129 Ft/kWh | Night: 249 Ft/kWh,False,True,EV Direct,Veszprém


In [374]:
# Check for remaining NULL values
print(f"Remaining missing counties: {geo_clean_df['county'].isna().sum()}")
print(f"Remaining missing cities: {geo_clean_df['city'].isna().sum()}")
print(f"Remaining missing postal codes: {geo_clean_df['postal_code'].isna().sum()}")

Remaining missing counties: 0
Remaining missing cities: 0
Remaining missing postal codes: 0


### 5. Data Type Conversion


Ensures all columns have appropriate data types for analysis (integers for counts, proper dates for timestamps).

In [375]:
# Check the data type of postal code column
geo_clean_df['postal_code'].dtype

dtype('O')

In [376]:
# Convert Postal code to integer
geo_clean_df['postal_code'] =  geo_clean_df['postal_code'].astype(int)


In [377]:
geo_clean_df['postal_code'].dtype

dtype('int64')

In [378]:
geo_clean_df['postal_code']

0      3000
1      1033
2      2310
3      8000
4      8241
       ... 
858    1115
859    1103
860    1031
861    1054
862    5742
Name: postal_code, Length: 863, dtype: int64

In [379]:
geo_clean_df['num_charging_points'].dtype

dtype('float64')

In [380]:
geo_clean_df['num_charging_points'].unique()

array([ 2., 12., 20.,  1., 16.,  7.,  4.,  3., 10.,  6.,  8.,  5., nan,
       18.])

In [381]:
geo_clean_df['num_charging_points'].isnull().any().sum()

np.int64(1)

In [382]:
geo_clean_df['num_charging_points'] = geo_clean_df['num_charging_points'] \
                                     .astype('Int64')

In [383]:
geo_clean_df['num_charging_points'].dtype

Int64Dtype()

In [384]:
geo_clean_df['num_charging_points']

0       2
1      12
2      20
3      12
4       2
       ..
858     2
859     2
860     2
861     2
862     1
Name: num_charging_points, Length: 863, dtype: Int64

In [385]:
geo_clean_df['num_charging_points'].sum()

np.int64(1843)

In [386]:
# Convert date columns
geo_clean_df['last_verified_date'] = pd.to_datetime(geo_clean_df['last_verified_date']).dt.date
geo_clean_df['creation_date'] = pd.to_datetime(geo_clean_df['creation_date']).dt.date

### 6. Usage Cost Data Cleaning


Standardizes the pricing information which comes in various formats (free, AC/DC pricing, time-based fees).

In [387]:
# Usage cost
geo_clean_df['usage_cost'].unique()

array([nan, '202 HUF/kWh', 'Day: 129 Ft/kWh | Night: 249 Ft/kWh',
       '225Ft/kW', '175 Ft/kWh', '225', '150 Ft/kWh', '120 Ft/kWh',
       '180 Ft/kWh', '170 HUF/kWh + 4 HUF/min', 'HUF280/kWh',
       '250 HUF/kWh', '160 Ft/kWh', '225 HUF/kWh.', '170 Ft/kWh',
       '143,- HUF/kWh ', 'DC 75 kW: 189Ft/kWh, AC 22 kW: 189Ft/kWh',
       'DC 100 kW: 169Ft/kWh ', '190', '299 Huf/kWh', '169,- HUF/kWh ',
       '164 Ft/kWh', '123,- Ft/kWh', 'DC 100 kW: 116Ft/kWh',
       '250,- Ft/kWh', '319 HUF/kWh', '200 Ft/kWh',
       '130 HUF/kWh + 100 HUF kényelmi díj', '150 huf/min',
       'DC 60 kW: 139Ft/kWh, AC 22 kW: 119Ft/kWh',
       'AC 22 kW: 109Ft/kWh, DC 25 kW: 112Ft/kWh', '129,- Ft/kWh ',
       '0,- Ft/kWh', '30,- Ft/kWh', '99,- Ft/kWh', ' 186,- Ft/kWh',
       '22kW = 109,- Ft/kWh; 7.2 kW = 99,- Ft/kWh; 3.6 kW = 89,- Ft/kWh',
       '22kW = 139,- Ft/kWh; 7.2 kW = 135,- Ft/kWh; 3.6 kW = 129,- Ft/kWh',
       '22kW = 109,- Ft/kWh; 7.2 kW = 99,- Ft/kWh',
       'AC 22 kW: 99Ft/kWh, DC 50 k

In [388]:
def clean_usage_cost(raw_prices: pd.Series) -> pd.DataFrame:
     """
    Clean and standardize EV charging station pricing data.
    
    Args:
        raw_prices: List of raw pricing strings
    
    Returns:
        DataFrame with cleaned pricing information
    """
     
     clean_prices = []

     for idx, price_str in enumerate(raw_prices):
          record = {
               'original_text': price_str,
               'is_free': False,
               'is_paid_unspecified': False,
               'is_inaccessible': False,
               'ac_price_huf_kwh': None,
               'dc_price_huf_kwh': None,
               'time_based_price_huf_min': None,
               'additional_fees': None,
               'notes': None
          }

          # Handle NaN and None values
          if pd.isna(price_str) or price_str is None:
               record['notes'] = 'Missing data'
               clean_prices.append(record)
               continue
          
          price_str = str(price_str).strip()

          price_lower = price_str.lower()

          # Check for free charging
          if any(keyword in price_lower for keyword in ['free', 'ingyenes', '0.- ft/kwh', '0 ft']):
               record['is_free'] = True
               record['notes'] = 'Free charging'
               clean_prices.append(record)
               continue
          
          # Check for inaccessible or special access
          if any(keyword in price_lower for keyword in ['not publicly accessible', 'tagoknak', 'klinika', 'hotel', 'guests']):
               record['is_inaccessible'] = True
               record['notes'] = 'Restricted access or special conditions'
               clean_prices.append(record)
               continue

          # Check for unspecified paid cervice
          if any(keyword in price_lower for keyword in ['paid']):
               record['is_paid_unspecified'] = True
               record['notes'] = 'Paid service - price not specified'
               clean_prices.append(record)
               continue

          # Exctract numeric prices using regex
          try:
               # Pattern for AC charging prices
               ac_match = re.search(r'ac[^:]*:\s*(\d+(?:[,.]\d+)?)\s*(?:ft|huf)/kwh', price_lower)
               if ac_match:
                    record['ac_price_huf_kwh'] = float(ac_match.group(1).replace(',', '.'))

               # Pattern for DC charging
               dc_match = re.search(r'dc[^:]*:\s*(\d+(?:[,.]\d+)?)\s*(?:ft|huf)/kwh', price_lower)
               if dc_match:
                    record['dc_price_huf_kwh'] = float(ac_match.group(1).replace(',', '.'))

               # Pattern for time-based pricing
               time_based_match = re.search(r'(\d+(?:[,.]\d+)?)\s*(?:ft|huf)/(?:min|perc)', price_lower)
               if time_based_match:
                    record['time_based_price_huf_min'] = float(time_based_match.group(1).replace(',', '.'))

               # When no AC/DC specified, try to extract general kWh price
               if not record['ac_price_huf_kwh'] and not record['dc_price_huf_kwh']:
                    general_match = re.search(r'(\d+(?:[,.]\d+)?)\s*[,-]*\s*(?:ft|huf)/kwh', price_lower)
                    if general_match:
                         price_value = float(general_match.group(1).replace(',', '.'))
                         # We add them to AC if not specified
                         record['ac_price_huf_kwh'] = price_value

               # Extract additional fees
               if any(keyword in price_lower for keyword in ['díj', 'fee', '+', 'after']):
                    record['additional_fees'] = 'Has additional fees or conditions'

          except (ValueError, AttributeError) as e:
               record['notes'] = f'Could not parce price: {str(e)}'


          # Adding some special notes for complex pricing
          if '|' in price_str or 'day:' in price_lower or 'night:' in price_lower:
               record['notes'] = 'Time-of-use-pricing'
          elif 'after' in price_lower:
               record['notes'] = 'Time-based additional charges apply'
          elif 'kw:' in price_lower:
               record['notes'] = 'Power-level specific pricing'

          clean_prices.append(record)

     return pd.DataFrame(clean_prices)


In [389]:
pricing_clean_df = clean_usage_cost(geo_clean_df['usage_cost'].tolist())

In [390]:
# Merging the two dataframes
price_clean_df = pd.concat([geo_clean_df, pricing_clean_df], axis=1)

In [391]:
for col in ['is_free', 'is_inaccessible', 'is_paid_unspecified']:
    price_clean_df[col] = price_clean_df[col].fillna(False).astype(bool)

In [392]:
# Display basic statistics
print("=== EV CHARGING PRICING DATA CLEANING RESULTS ===\n")

print(f"Total records processed: {len(price_clean_df)}")
print(f"Free charging stations: {price_clean_df['is_free'].sum()}")
print(f"Restricted access stations: {price_clean_df['is_inaccessible'].sum()}")
print(f"Paid but unspecified price: {price_clean_df['is_paid_unspecified'].sum()}")

print("\n=== PRICING STATISTICS ===")

# AC Charging Statistics
ac_prices = price_clean_df['ac_price_huf_kwh'].dropna()
if len(ac_prices) > 0:
    print(f"\nAC Charging (HUF/kWh):")
    print(f"  Count: {len(ac_prices)}")
    print(f"  Min: {ac_prices.min():.0f}")
    print(f"  Max: {ac_prices.max():.0f}")
    print(f"  Average: {ac_prices.mean():.0f}")
    print(f"  Median: {ac_prices.median():.0f}")

# DC Charging Statistics
dc_prices = price_clean_df['dc_price_huf_kwh'].dropna()
if len(dc_prices) > 0:
    print(f"\nDC Charging (HUF/kWh):")
    print(f"  Count: {len(dc_prices)}")
    print(f"  Min: {dc_prices.min():.0f}")
    print(f"  Max: {dc_prices.max():.0f}")
    print(f"  Average: {dc_prices.mean():.0f}")
    print(f"  Median: {dc_prices.median():.0f}")

# Time-based pricing
time_prices = price_clean_df['time_based_price_huf_min'].dropna()
if len(time_prices) > 0:
    print(f"\nTime-based Charging (HUF/min):")
    print(f"  Count: {len(time_prices)}")
    print(f"  Min: {time_prices.min():.0f}")
    print(f"  Max: {time_prices.max():.0f}")
    print(f"  Average: {time_prices.mean():.0f}")

print("\n=== SAMPLE OF CLEANED DATA ===")
# Show first 10 non-free, accessible records
sample_data = price_clean_df[
    (~price_clean_df['is_free']) & 
    (~price_clean_df['is_inaccessible']) & 
    (~price_clean_df['is_paid_unspecified'])
].head(10)

for idx, row in sample_data.iterrows():
    print(f"\nOriginal: {row['original_text']}")
    if row['ac_price_huf_kwh']:
        print(f"  AC Price: {row['ac_price_huf_kwh']:.0f} HUF/kWh")
    if row['dc_price_huf_kwh']:
        print(f"  DC Price: {row['dc_price_huf_kwh']:.0f} HUF/kWh")
    if row['time_based_price_huf_min']:
        print(f"  Time-based: {row['time_based_price_huf_min']:.0f} HUF/min")
    if row['additional_fees']:
        print(f"  Note: {row['additional_fees']}")

# Export options
print("\n=== EXPORT OPTIONS ===")
print("# Save to CSV")
price_clean_df.to_csv('cleaned_ev_pricing.csv', index=False)

print("\n# Create summary table")
summary_stats = {
    'AC_Charging_Available': len(ac_prices),
    'AC_Average_Price_HUF_kWh': ac_prices.mean() if len(ac_prices) > 0 else None,
    'DC_Charging_Available': len(dc_prices),
    'DC_Average_Price_HUF_kWh': dc_prices.mean() if len(dc_prices) > 0 else None,
    'Free_Stations': price_clean_df['is_free'].sum(),
    'Restricted_Access': price_clean_df['is_inaccessible'].sum()
}

print(f"Summary: {summary_stats}")

# Show the cleaned DataFrame structure
print(f"\n=== DATAFRAME INFO ===")
print(f"Shape: {price_clean_df.shape}")
print(f"Columns: {list(price_clean_df.columns)}")
print(f"\nColumn data types:")
for col in price_clean_df.columns:
    print(f"  {col}: {price_clean_df[col].dtype}")

=== EV CHARGING PRICING DATA CLEANING RESULTS ===

Total records processed: 863
Free charging stations: 345
Restricted access stations: 3
Paid but unspecified price: 1

=== PRICING STATISTICS ===

AC Charging (HUF/kWh):
  Count: 162
  Min: 0
  Max: 319
  Average: 124
  Median: 109

DC Charging (HUF/kWh):
  Count: 12
  Min: 99
  Max: 189
  Average: 114
  Median: 109

Time-based Charging (HUF/min):
  Count: 63
  Min: 1
  Max: 150
  Average: 22

=== SAMPLE OF CLEANED DATA ===

Original: nan
  AC Price: nan HUF/kWh
  DC Price: nan HUF/kWh
  Time-based: nan HUF/min

Original: 202 HUF/kWh
  AC Price: 202 HUF/kWh
  DC Price: nan HUF/kWh
  Time-based: nan HUF/min

Original: 202 HUF/kWh
  AC Price: 202 HUF/kWh
  DC Price: nan HUF/kWh
  Time-based: nan HUF/min

Original: 202 HUF/kWh
  AC Price: 202 HUF/kWh
  DC Price: nan HUF/kWh
  Time-based: nan HUF/min

Original: Day: 129 Ft/kWh | Night: 249 Ft/kWh
  AC Price: 129 HUF/kWh
  DC Price: nan HUF/kWh
  Time-based: nan HUF/min

Original: 225Ft/kW
 

### 7. Operator Data Cleaning

Standardizes operator names and handles missing values.

In [393]:
price_clean_df['operator'].unique()

array(['Mobiliti.hu', 'Tesla (including non-tesla)', 'EV Direct',
       'Curb (Hungary)', 'Elektromotive (UK)', '(Unknown Operator)',
       'DV Parking (HU)', 'Ionity', 'Elektro Profi Mobility (HU)', 'TEA.',
       nan, 'MOL', 'Gyorstöltők', 'E.ON (HU)',
       '(Business Owner at Location)', 'NKM mobiliti', 'Lidl', 'Volthero',
       'E-Mobi', 'Optimum Way', 'Innogy SE (RWE eMobility)',
       '(Private Residence/Individual)', 'Tesla (Tesla-only charging)',
       'MVM Partner Zrt.'], dtype=object)

In [394]:
def clean_operator_data(price_clean_df):
    """
    Clean and standardize operator names in the dataset.
    
    Args:
        price_clean_df: The cleaned DataFrame containing pricing information that we want to add operator cleaning to
        
    Returns:
        DataFrame with cleaned operator names and additional operator-related columns
    """

    # Create a copy
    operator_clean_df = price_clean_df.copy()

    # Replace NaN values and Unknown
    operator_clean_df['operator'] = operator_clean_df['operator'].fillna('Unknown')

    # Standardize operators
    operator_clean_df['operator'] = operator_clean_df['operator'].replace({
        'Tesla (including non-tesla)': 'Tesla',
        'Tesla (Tesla-only charging)': 'Tesla',
        '(Unknown Operator)': 'Unknown',
        '(Business Owner at Location)': 'Business Owner',
        '(Private Residence/Individual)': 'Private Individual'
    })

    # Remove country codes and domains
    operator_clean_df['operator'] = operator_clean_df['operator'].str.replace(r' \(HU\)', '', regex=True)
    operator_clean_df['operator'] = operator_clean_df['operator'].str.replace(r' \(UK\)', '', regex=True)
    operator_clean_df['operator'] = operator_clean_df['operator'].str.replace(r' \(Hungary\)', '', regex=True)
    operator_clean_df['operator'] = operator_clean_df['operator'].str.replace(r' \(RWE eMobility\)', '', regex=True)
    operator_clean_df['operator'] = operator_clean_df['operator'].str.replace(r'\.', '', regex=True)
    operator_clean_df['operator'] = operator_clean_df['operator'].str.replace(r'\.hu$', '', regex=True) 

    # Create a separate Tesla type column to distinguish Tesla charging types
    operator_clean_df['tesla_type'] = operator_clean_df['operator'].map({
        'Tesla (including non-tesla)': 'Multi-brand',
        'Tesla (Tesla-only charging)': 'Tesla-only'
    })

    return operator_clean_df

In [395]:
operator_clean_df = clean_operator_data(price_clean_df)

In [396]:
operator_clean_df['operator'].unique()

array(['Mobilitihu', 'Tesla', 'EV Direct', 'Curb', 'Elektromotive',
       'Unknown', 'DV Parking', 'Ionity', 'Elektro Profi Mobility', 'TEA',
       'MOL', 'Gyorstöltők', 'EON', 'Business Owner', 'NKM mobiliti',
       'Lidl', 'Volthero', 'E-Mobi', 'Optimum Way', 'Innogy SE',
       'Private Individual', 'MVM Partner Zrt'], dtype=object)

In [397]:
unknown_count = (operator_clean_df['operator'] == 'Unknown').sum()
print(f'Number of unknown operators: {unknown_count}')

Number of unknown operators: 76


In [398]:
# Creating a final_df, excluding the connections
final_df = operator_clean_df.drop(columns=['connections'])

#### 8. Connections Data Cleaning

The connections data contains nested information about charger types and specifications. This extracts it into a separate, normalized dataframe.

In [399]:
operator_clean_df['connections'].unique()

array(["[{'ID': 603425, 'ConnectionTypeID': 33, 'ConnectionType': {'FormalName': 'IEC 62196-3 Configuration FF', 'IsDiscontinued': False, 'IsObsolete': False, 'ID': 33, 'Title': 'CCS (Type 2)'}, 'Reference': None, 'StatusTypeID': 50, 'StatusType': {'IsOperational': True, 'IsUserSelectable': True, 'ID': 50, 'Title': 'Operational'}, 'LevelID': None, 'Level': None, 'Amps': None, 'Voltage': None, 'PowerKW': None, 'CurrentTypeID': None, 'CurrentType': None, 'Quantity': 1, 'Comments': None}, {'ID': 604597, 'ConnectionTypeID': 2, 'ConnectionType': {'FormalName': 'IEC 62196-3 Configuration AA', 'IsDiscontinued': None, 'IsObsolete': None, 'ID': 2, 'Title': 'CHAdeMO'}, 'Reference': None, 'StatusTypeID': 50, 'StatusType': {'IsOperational': True, 'IsUserSelectable': True, 'ID': 50, 'Title': 'Operational'}, 'LevelID': None, 'Level': None, 'Amps': None, 'Voltage': None, 'PowerKW': None, 'CurrentTypeID': None, 'CurrentType': None, 'Quantity': 1, 'Comments': None}]",
       "[{'ID': 603422, 'Connectio

In [400]:
import ast

def clean_connections_data(connection_series):
    """
    Parameters:
    connections_series: pandas Series containing string representations of connection data
    
    Returns:
    tuple: (cleaned_dataframe, summary_stats)
    """

    all_connections = []
    station_id = 0

    print("🔧 Starting data cleaning process...")

    for idx, connection_str in enumerate(connection_series):
        if pd.isna(connection_str):
            continue

        try:
            # Parse the string representation of the list
            if isinstance (connection_str, str):
                # Handle the case where it might be a string representation
                connections_list = ast.literal_eval(connection_str)
            else:
                connections_list = connection_str

            
            # Extract data from each connection in the list
            for conn in connections_list:
                connection_data = {
                    'station_id': station_id,
                    'connection_id': conn.get('ID'),
                    'connection_type_id': conn.get('ConnectionTypeID'),
                    'connection_type_title': conn.get('ConnectionType', {}).get('Title') if conn.get('ConnectionType') else None,
                    'connection_formal_name': conn.get('ConnectionType', {}).get('FormalName') if conn.get('ConnectionType') else None,
                    'is_discontinued': conn.get('ConnectionType', {}).get('IsDiscontinued') if conn.get('ConnectionType') else None,
                    'is_obsolete': conn.get('ConnectionType', {}).get('IsObsolete') if conn.get('ConnectionType') else None,
                    'reference': conn.get('Reference'),
                    'status_type_id': conn.get('StatusTypeID'),
                    'status_title': conn.get('StatusType', {}).get('Title') if conn.get('StatusType') else None,
                    'is_operational': conn.get('StatusType', {}).get('IsOperational') if conn.get('StatusType') else None,
                    'is_user_selectable': conn.get('StatusType', {}).get('IsUserSelectable') if conn.get('StatusType') else None,
                    'level_id': conn.get('LevelID'),
                    'level_title': conn.get('Level', {}).get('Title') if conn.get('Level') else None,
                    'level_comments': conn.get('Level', {}).get('Comments') if conn.get('Level') else None,
                    'is_fast_charge_capable': conn.get('Level', {}).get('IsFastChargeCapable') if conn.get('Level') else None,
                    'amps': conn.get('Amps'),
                    'voltage': conn.get('Voltage'),
                    'power_kw': conn.get('PowerKW'),
                    'current_type_id': conn.get('CurrentTypeID'),
                    'current_type_title': conn.get('CurrentType', {}).get('Title') if conn.get('CurrentType') else None,
                    'current_type_description': conn.get('CurrentType', {}).get('Description') if conn.get('CurrentType') else None,
                    'quantity': conn.get('Quantity'),
                    'comments': conn.get('Comments')
                }
                all_connections.append(connection_data)
            
            station_id += 1
            
        except (ValueError, SyntaxError, TypeError) as e:
            print(f"⚠️  Error parsing row {idx}: {str(e)[:100]}...")
            continue
            
    df_clean_connections = pd.DataFrame(all_connections)

    print(f"✅ Successfully processed {len(df_clean_connections)} connections from {station_id} stations")

    return df_clean_connections

In [401]:
# Create separate connections dataframe
connections_df = clean_connections_data(operator_clean_df['connections'])

🔧 Starting data cleaning process...
✅ Successfully processed 1358 connections from 863 stations


In [402]:
def analyze_connections_data(df):
    """
    Perform comprehensive analysis of the cleaned connections data
    """
    
    print("\n" + "="*60)
    print("📊 COMPREHENSIVE DATA ANALYSIS")
    print("="*60)
    
    # Basic statistics
    print(f"\n📈 BASIC STATISTICS:")
    print(f"Total connections: {len(df)}")
    print(f"Total stations: {df['station_id'].nunique()}")
    print(f"Average connections per station: {len(df) / df['station_id'].nunique():.2f}")
    
    # Connection types analysis
    print(f"\n🔌 CONNECTION TYPES:")
    conn_types = df['connection_type_title'].value_counts()
    for conn_type, count in conn_types.head(10).items():
        percentage = (count / len(df)) * 100
        print(f"  {conn_type}: {count} ({percentage:.1f}%)")
    
    # Power levels analysis
    print(f"\n⚡ POWER LEVELS:")
    power_levels = df['level_title'].value_counts()
    for level, count in power_levels.items():
        percentage = (count / len(df)) * 100
        print(f"  {level}: {count} ({percentage:.1f}%)")
    
    # Power capacity analysis
    print(f"\n🔋 POWER CAPACITY ANALYSIS:")
    power_stats = df['power_kw'].describe()
    print(f"  Average power: {power_stats['mean']:.1f} kW")
    print(f"  Max power: {power_stats['max']:.0f} kW")
    print(f"  Min power: {power_stats['min']:.0f} kW")
    
    # Current type analysis
    print(f"\n🔄 CURRENT TYPES:")
    current_types = df['current_type_title'].value_counts()
    for curr_type, count in current_types.items():
        percentage = (count / len(df)) * 100
        print(f"  {curr_type}: {count} ({percentage:.1f}%)")
    
    # Status analysis
    print(f"\n✅ OPERATIONAL STATUS:")
    operational_count = df['is_operational'].sum()
    print(f"  Operational: {operational_count} ({(operational_count/len(df)*100):.1f}%)")
    print(f"  Non-operational: {len(df) - operational_count} ({((len(df) - operational_count)/len(df)*100):.1f}%)")
    
    # Fast charging capability
    print(f"\n🚀 FAST CHARGING CAPABILITY:")
    fast_charge_count = df['is_fast_charge_capable'].sum()
    total_with_info = df['is_fast_charge_capable'].notna().sum()
    if total_with_info > 0:
        print(f"  Fast charge capable: {fast_charge_count} ({(fast_charge_count/total_with_info*100):.1f}%)")
    
    # Quantity analysis
    print(f"\n📊 QUANTITY DISTRIBUTION:")
    quantity_stats = df['quantity'].describe()
    print(f"  Average quantity per connection: {quantity_stats['mean']:.1f}")
    print(f"  Max quantity: {quantity_stats['max']:.0f}")
    print(f"  Total charging points: {df['quantity'].sum():.0f}")

In [403]:
def create_summary_dataframes(df):
    """
    Create summary dataframes for different aspects of the data
    """
    
    # Connection type summary
    conn_summary = df.groupby(['connection_type_title', 'level_title']).agg({
        'connection_id': 'count',
        'quantity': 'sum',
        'power_kw': ['mean', 'max', 'min'],
        'is_operational': 'sum'
    }).round(2)
    
    # Station summary
    station_summary = df.groupby('station_id').agg({
        'connection_id': 'count',
        'quantity': 'sum',
        'power_kw': 'max',
        'connection_type_title': lambda x: ', '.join(x.unique()),
        'is_operational': 'all'
    })
    
    # Power range categories
    df_power = df.copy()
    df_power['power_category'] = pd.cut(df_power['power_kw'], 
                                       bins=[0, 7, 22, 50, 150, 1000], 
                                       labels=['Slow (≤7kW)', 'Medium (7-22kW)', 
                                              'Fast (22-50kW)', 'Rapid (50-150kW)', 
                                              'Ultra-rapid (>150kW)'])
    
    power_summary = df_power.groupby('power_category').agg({
        'connection_id': 'count',
        'quantity': 'sum',
        'power_kw': ['mean', 'max'],
        'is_fast_charge_capable': 'sum'
    }).round(2)
    
    return conn_summary, station_summary, power_summary


In [404]:
analyze_connections_data(connections_df)


📊 COMPREHENSIVE DATA ANALYSIS

📈 BASIC STATISTICS:
Total connections: 1358
Total stations: 862
Average connections per station: 1.58

🔌 CONNECTION TYPES:
  Type 2 (Socket Only): 729 (53.7%)
  CCS (Type 2): 223 (16.4%)
  CHAdeMO: 210 (15.5%)
  Type 2 (Tethered Connector) : 115 (8.5%)
  CEE 7/4 - Schuko - Type F: 46 (3.4%)
  Tesla (Model S/X): 8 (0.6%)
  CEE 5 Pin: 8 (0.6%)
  NACS / Tesla Supercharger: 6 (0.4%)
  IEC 60309 5-pin: 5 (0.4%)
  CCS (Type 1): 2 (0.1%)

⚡ POWER LEVELS:
  Level 2 : Medium (Over 2kW): 850 (62.6%)
  Level 3:  High (Over 40kW): 492 (36.2%)
  Level 1 : Low (Under 2kW): 13 (1.0%)

🔋 POWER CAPACITY ANALYSIS:
  Average power: 33.3 kW
  Max power: 350 kW
  Min power: 2 kW

🔄 CURRENT TYPES:
  AC (Three-Phase): 817 (60.2%)
  DC: 430 (31.7%)
  AC (Single-Phase): 81 (6.0%)

✅ OPERATIONAL STATUS:
  Operational: 1302 (95.9%)
  Non-operational: 56 (4.1%)

🚀 FAST CHARGING CAPABILITY:
  Fast charge capable: 492 (36.3%)

📊 QUANTITY DISTRIBUTION:
  Average quantity per connectio

In [405]:
 # Display sample of cleaned data
connections_df.head()

,station_id,connection_id,connection_type_id,connection_type_title,connection_formal_name,is_discontinued,is_obsolete,reference,status_type_id,status_title,...,level_comments,is_fast_charge_capable,amps,voltage,power_kw,current_type_id,current_type_title,current_type_description,quantity,comments
0,0,603425,33,CCS (Type 2),IEC 62196-3 Configuration FF,False,False,None,50.0,Operational,...,None,None,NaN,NaN,NaN,NaN,None,None,1.0,None
1,0,604597,2,CHAdeMO,IEC 62196-3 Configuration AA,None,None,None,50.0,Operational,...,None,None,NaN,NaN,NaN,NaN,None,None,1.0,None
2,1,603422,33,CCS (Type 2),IEC 62196-3 Configuration FF,False,False,None,50.0,Operational,...,40KW and Higher,True,NaN,NaN,250.0,30.0,DC,Direct Current,12.0,None
3,2,603421,33,CCS (Type 2),IEC 62196-3 Configuration FF,False,False,None,50.0,Operational,...,40KW and Higher,True,NaN,NaN,250.0,30.0,DC,Direct Current,20.0,None
4,3,603420,33,CCS (Type 2),IEC 62196-3 Configuration FF,False,False,None,50.0,Operational,...,40KW and Higher,True,NaN,NaN,250.0,30.0,DC,Direct Current,12.0,None


In [406]:
print(f"\n📝 COLUMNS IN CLEANED CONNECTIONS DATASET:")
for i, col in enumerate(connections_df.columns, 1):
    print(f"  {i:2d}. {col}")


📝 COLUMNS IN CLEANED CONNECTIONS DATASET:
   1. station_id
   2. connection_id
   3. connection_type_id
   4. connection_type_title
   5. connection_formal_name
   6. is_discontinued
   7. is_obsolete
   8. reference
   9. status_type_id
  10. status_title
  11. is_operational
  12. is_user_selectable
  13. level_id
  14. level_title
  15. level_comments
  16. is_fast_charge_capable
  17. amps
  18. voltage
  19. power_kw
  20. current_type_id
  21. current_type_title
  22. current_type_description
  23. quantity
  24. comments


### 9. Final Cleaning and Validation

Quality checks to ensure the cleaning process hasn't lost important data or introduced errors.

In [409]:
def validate_cleaning_results(raw_df, geo_clean_df, price_clean_df, operator_clean_df, connections_df):
    """
    Comprehensive validation of the cleaning process by comparing original and cleaned datasets.
    
    Parameters:
    - raw_df: Original DataFrame from API
    - geo_clean_df: DataFrame after geographic cleaning
    - price_clean_df: Cleaned stations DataFrame with pricing information
    - operator_clean_df: Cleaned stations DataFrame with operator information
    - connections_df: Cleaned connections DataFrame
    
    Returns:
    - Dictionary containing validation results and metrics
    - Prints detailed validation report
    """
    
    print("="*80)
    print("🔍 FINAL DATA VALIDATION REPORT")
    print("="*80)
    
    # Create a results dictionary to store metrics
    validation_results = {}
    
    # 1. Record Count Validation
    validation_results['original_count'] = len(raw_df)
    validation_results['geo_cleaned_count'] = len(geo_clean_df)
    validation_results['price_cleaned_count'] = len(price_clean_df)
    validation_results['operator_cleaned_count'] = len(operator_clean_df)
    validation_results['connections_count'] = len(connections_df)
    
    print(f"\n📊 RECORD COUNTS THROUGH CLEANING PIPELINE:")
    print(f"  Original stations: {validation_results['original_count']}")
    print(f"  After geo cleaning: {validation_results['geo_cleaned_count']} ({(validation_results['geo_cleaned_count']/validation_results['original_count'])*100:.1f}%)")
    print(f"  After pricing cleaning: {validation_results['price_cleaned_count']}")
    print(f"  After operator cleaning: {validation_results['operator_cleaned_count']}")
    print(f"  Individual connections: {validation_results['connections_count']}")
    
    # 2. Geographic Data Validation
    validation_results['original_geo_complete'] = raw_df[['AddressInfo.Latitude', 'AddressInfo.Longitude']].notna().all(axis=1).sum()
    validation_results['cleaned_geo_complete'] = operator_clean_df[['latitude', 'longitude']].notna().all(axis=1).sum()
    validation_results['unique_counties'] = operator_clean_df['county'].nunique()
    
    print("\n🌍 GEOGRAPHIC DATA VALIDATION:")
    print(f"  Stations with complete geo coordinates:")
    print(f"    Original: {validation_results['original_geo_complete']} ({(validation_results['original_geo_complete']/validation_results['original_count'])*100:.1f}%)")
    print(f"    Cleaned: {validation_results['cleaned_geo_complete']} ({(validation_results['cleaned_geo_complete']/validation_results['operator_cleaned_count'])*100:.1f}%)")
    print(f"  Unique standardized counties: {validation_results['unique_counties']}")
    
    # 3. Pricing Data Validation
    validation_results['free_stations'] = operator_clean_df['is_free'].sum()
    validation_results['paid_stations'] = (~operator_clean_df['is_free']).sum()
    validation_results['inaccessible'] = operator_clean_df['is_inaccessible'].sum()
    validation_results['has_price_info'] = (operator_clean_df['ac_price_huf_kwh'].notna() | 
                                         operator_clean_df['dc_price_huf_kwh'].notna() | 
                                         operator_clean_df['time_based_price_huf_min'].notna()).sum()
    
    print("\n💰 PRICING DATA VALIDATION:")
    print(f"  Free stations: {validation_results['free_stations']} ({(validation_results['free_stations']/validation_results['operator_cleaned_count'])*100:.1f}%)")
    print(f"  Paid stations: {validation_results['paid_stations']} ({(validation_results['paid_stations']/validation_results['operator_cleaned_count'])*100:.1f}%)")
    print(f"  Inaccessible stations: {validation_results['inaccessible']} ({(validation_results['inaccessible']/validation_results['operator_cleaned_count'])*100:.1f}%)")
    print(f"  Paid stations with price info: {validation_results['has_price_info']} ({(validation_results['has_price_info']/validation_results['paid_stations'])*100:.1f}%)")
    
    # 4. Operator Data Validation
    validation_results['operator_categories'] = operator_clean_df['operator'].value_counts().to_dict()
    validation_results['unknown_operators'] = (operator_clean_df['operator'] == 'Unknown').sum()
    
    print("\n🏢 OPERATOR DATA VALIDATION:")
    print("  Operator categories:")
    for category, count in validation_results['operator_categories'].items():
        print(f"    {category}: {count} ({(count/validation_results['operator_cleaned_count'])*100:.1f}%)")
    print(f"  Unknown operators: {validation_results['unknown_operators']} ({(validation_results['unknown_operators']/validation_results['operator_cleaned_count'])*100:.1f}%)")
    
    # 5. Connections Data Validation
    validation_results['top_connection_types'] = connections_df['connection_type_title'].value_counts().head(5).to_dict()
    validation_results['power_stats'] = connections_df['power_kw'].describe().to_dict()
    validation_results['operational_connections'] = connections_df['is_operational'].sum()
    
    print("\n🔌 CONNECTIONS DATA VALIDATION:")
    print("  Top 5 connection types:")
    for conn_type, count in validation_results['top_connection_types'].items():
        print(f"    {conn_type}: {count} ({(count/validation_results['connections_count'])*100:.1f}%)")
    
    print("\n  Power distribution (kW):")
    print(f"    Mean: {validation_results['power_stats']['mean']:.1f}")
    print(f"    Min: {validation_results['power_stats']['min']:.1f}")
    print(f"    Max: {validation_results['power_stats']['max']:.1f}")
    
    print(f"\n  Operational connections: {validation_results['operational_connections']} ({(validation_results['operational_connections']/validation_results['connections_count'])*100:.1f}%)")
    
    # 6. Final Data Quality Score
    quality_score = (
        (validation_results['cleaned_geo_complete'] / validation_results['operator_cleaned_count']) * 0.2 +
        (validation_results['has_price_info'] / max(1, validation_results['paid_stations'])) * 0.3 +
        (1 - (validation_results['unknown_operators'] / validation_results['operator_cleaned_count'])) * 0.2 +
        (validation_results['operational_connections'] / validation_results['connections_count']) * 0.3
    ) * 100
    
    print("\n⭐ OVERALL DATA QUALITY SCORE:")
    print(f"  {quality_score:.1f}/100 based on completeness of geo, pricing, operator, and operational status data")
    
    validation_results['quality_score'] = quality_score
    
    return validation_results

In [410]:
validate_cleaning_results(raw_df, geo_clean_df, price_clean_df, operator_clean_df, connections_df)

🔍 FINAL DATA VALIDATION REPORT

📊 RECORD COUNTS THROUGH CLEANING PIPELINE:
  Original stations: 863
  After geo cleaning: 863 (100.0%)
  After pricing cleaning: 863
  After operator cleaning: 863
  Individual connections: 1358

🌍 GEOGRAPHIC DATA VALIDATION:
  Stations with complete geo coordinates:
    Original: 863 (100.0%)
    Cleaned: 863 (100.0%)
  Unique standardized counties: 20

💰 PRICING DATA VALIDATION:
  Free stations: 345 (40.0%)
  Paid stations: 518 (60.0%)
  Inaccessible stations: 3 (0.3%)
  Paid stations with price info: 205 (39.6%)

🏢 OPERATOR DATA VALIDATION:
  Operator categories:
    EON: 192 (22.2%)
    Business Owner: 174 (20.2%)
    E-Mobi: 136 (15.8%)
    Unknown: 76 (8.8%)
    TEA: 66 (7.6%)
    MOL: 56 (6.5%)
    NKM mobiliti: 47 (5.4%)
    Mobilitihu: 21 (2.4%)
    MVM Partner Zrt: 20 (2.3%)
    Tesla: 19 (2.2%)
    Curb: 13 (1.5%)
    Optimum Way: 10 (1.2%)
    DV Parking: 9 (1.0%)
    Ionity: 6 (0.7%)
    Private Individual: 6 (0.7%)
    EV Direct: 3 (0.3%)
 

{'original_count': 863,
 'geo_cleaned_count': 863,
 'price_cleaned_count': 863,
 'operator_cleaned_count': 863,
 'connections_count': 1358,
 'original_geo_complete': np.int64(863),
 'cleaned_geo_complete': np.int64(863),
 'unique_counties': 20,
 'free_stations': np.int64(345),
 'paid_stations': np.int64(518),
 'inaccessible': np.int64(3),
 'has_price_info': np.int64(205),
 'operator_categories': {'EON': 192,
  'Business Owner': 174,
  'E-Mobi': 136,
  'Unknown': 76,
  'TEA': 66,
  'MOL': 56,
  'NKM mobiliti': 47,
  'Mobilitihu': 21,
  'MVM Partner Zrt': 20,
  'Tesla': 19,
  'Curb': 13,
  'Optimum Way': 10,
  'DV Parking': 9,
  'Ionity': 6,
  'Private Individual': 6,
  'EV Direct': 3,
  'Elektro Profi Mobility': 2,
  'Elektromotive': 2,
  'Volthero': 2,
  'Lidl': 1,
  'Innogy SE': 1,
  'Gyorstöltők': 1},
 'unknown_operators': np.int64(76),
 'top_connection_types': {'Type 2 (Socket Only)': 729,
  'CCS (Type 2)': 223,
  'CHAdeMO': 210,
  'Type 2 (Tethered Connector) ': 115,
  'CEE 7/4 - S

### 10. Data Export

In [414]:
print("\n=== EXPORTING CLEANED DATA ===")

# Export cleaned data
final_df.to_csv('final_cleaned_ev_stations.csv', index=False)

 # Connections data    
connections_df.to_csv('connections_data.csv', index=False)


# Display final information
print("\n=== FINAL DATASET INFORMATION ===")
print("Stations Data:")
print(price_clean_df.info())
print("\nConnections Data:")
print(connections_df.info())


=== EXPORTING CLEANED DATA ===

=== FINAL DATASET INFORMATION ===
Stations Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863 entries, 0 to 862
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        863 non-null    int64  
 1   latitude                  863 non-null    float64
 2   longitude                 863 non-null    float64
 3   city                      863 non-null    object 
 4   postal_code               863 non-null    int64  
 5   country                   863 non-null    object 
 6   access_comments           242 non-null    object 
 7   connections               863 non-null    object 
 8   num_charging_points       862 non-null    Int64  
 9   is_operational            863 non-null    bool   
 10  last_verified_date        863 non-null    object 
 11  creation_date             863 non-null    object 
 12  usage_cost                632 non-null

In [415]:
final_df.head()

,ID,latitude,longitude,city,postal_code,country,access_comments,num_charging_points,is_operational,last_verified_date,...,original_text,is_free,is_paid_unspecified,is_inaccessible,ac_price_huf_kwh,dc_price_huf_kwh,time_based_price_huf_min,additional_fees,notes,tesla_type
0,313310,47.677106,19.667077,Hatvan,3000,Hungary,NaN,2,True,2025-06-16,...,NaN,False,False,False,NaN,NaN,NaN,None,Missing data,NaN
1,313307,47.561080,19.049120,Budapest,1033,Hungary,NaN,12,True,2025-06-07,...,202 HUF/kWh,False,False,False,202.0,NaN,NaN,None,None,NaN
2,313306,47.383440,19.026830,Szigetszentmiklós,2310,Hungary,NaN,20,True,2025-06-07,...,202 HUF/kWh,False,False,False,202.0,NaN,NaN,None,None,NaN
3,313305,47.149650,18.431330,Székesfehérvár,8000,Hungary,NaN,12,True,2025-06-07,...,202 HUF/kWh,False,False,False,202.0,NaN,NaN,None,None,NaN
4,312979,46.926144,17.837953,Aszófő,8241,Hungary,NaN,2,True,2025-06-11,...,Day: 129 Ft/kWh | Night: 249 Ft/kWh,False,False,False,129.0,NaN,NaN,None,Time-of-use-pricing,NaN


In [416]:
connections_df.head()

,station_id,connection_id,connection_type_id,connection_type_title,connection_formal_name,is_discontinued,is_obsolete,reference,status_type_id,status_title,...,level_comments,is_fast_charge_capable,amps,voltage,power_kw,current_type_id,current_type_title,current_type_description,quantity,comments
0,0,603425,33,CCS (Type 2),IEC 62196-3 Configuration FF,False,False,None,50.0,Operational,...,None,None,NaN,NaN,NaN,NaN,None,None,1.0,None
1,0,604597,2,CHAdeMO,IEC 62196-3 Configuration AA,None,None,None,50.0,Operational,...,None,None,NaN,NaN,NaN,NaN,None,None,1.0,None
2,1,603422,33,CCS (Type 2),IEC 62196-3 Configuration FF,False,False,None,50.0,Operational,...,40KW and Higher,True,NaN,NaN,250.0,30.0,DC,Direct Current,12.0,None
3,2,603421,33,CCS (Type 2),IEC 62196-3 Configuration FF,False,False,None,50.0,Operational,...,40KW and Higher,True,NaN,NaN,250.0,30.0,DC,Direct Current,20.0,None
4,3,603420,33,CCS (Type 2),IEC 62196-3 Configuration FF,False,False,None,50.0,Operational,...,40KW and Higher,True,NaN,NaN,250.0,30.0,DC,Direct Current,12.0,None
